<a href="https://colab.research.google.com/github/larissapoghosyan/Capstone_Project/blob/main/ELMo_Embedings_both_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install allennlp==0.9.0
!pip install flair
!pip install sacremoses

In [ ]:
from flair.data import Sentence
from flair.embeddings import ELMoEmbeddings, DocumentPoolEmbeddings
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import warnings
from tqdm import tqdm
import torch
import h5py
import time

IMDb

In [ ]:
data = pd.read_csv('/content/IMDb_Reviews.csv',engine='python', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 27866: unexpected end of data


In [ ]:
dataset_prc20 = data.sample(int(data.shape[0]*0.2),
                            random_state=22)

Intent Classification Dataset

In [ ]:
data_intent = pd.read_csv('/content/outofscope-intent-classification-dataset.csv')

In [ ]:
le = LabelEncoder()
np.random.seed(25)
label_sample = np.random.choice(data_intent['label'].unique(),
                                size=(10),
                                replace=False)
label_sample

array(['what_can_i_ask_you', 'what_are_your_hobbies', 'order',
       'change_accent', 'new_card', 'damaged_card', 'do_you_have_pets',
       'last_maintenance', 'reminder_update', 'bill_balance'],
      dtype=object)

In [ ]:
label_col_intent = np.array(data_intent['label']).reshape(data_intent.shape[0],1)

In [ ]:
data_sample = data[data['label'].isin(label_sample)]
data_sample['label'] = le.fit_transform(data_sample['label'])

## ELMo embeddings

In [ ]:
embeddings = []
token_embedding = ELMoEmbeddings()
document_embedding = DocumentPoolEmbeddings([token_embedding])
# dataset_prc20 for IMDb and data_sample for Intent Clf dataset
for sent in tqdm(dataset_prc20.iloc[:,0]):
  sentence = Sentence(sent)
  document_embedding.embed(sentence)
  embeddings.append(sentence.embedding)

100%|██████████| 1500/1500 [00:35<00:00, 42.17it/s]


In [ ]:
embeddings[0].shape

torch.Size([3072])

In [ ]:
device = 'cpu'
# stack the embeddings along axis 0  to obtain an embeddings tensor of size (data.shape[0], embeddings_size)
embeddings_tensor = torch.stack(embeddings, dim=0).to(device)

In [ ]:
embeddings_tensor.shape

torch.Size([1500, 3072])

ELMo Embeddings for the IMDb Dataset

In [ ]:
hf_elmo = h5py.File('elmo_imdb_clf.h5', 'w')
elmo_np = embeddings_tensor.numpy()
elmo_imdb_hf = hf_elmo.create_dataset('elmo_imdb_clf', data=elmo_np)
hf_elmo.close()

ELMo Embeddings for the Intnet CLF Dataset

In [ ]:
hf_elmo = h5py.File('elmo_intent_clf.h5', 'w')
elmo_np = embeddings_tensor.numpy()
elmo_imdb_hf = hf_elmo.create_dataset('elmo_intent_clf', data=elmo_np)
hf_elmo.close()

## Measuring Inference Time for 1 datapoint on GPU (IMDb dataset)
 (from the final results ELMo is one of the best performing models, we check the inference time for comparison

In [ ]:
# We sample 1000 data points randomly and then run the algorithm on those points
data_1000_sample = data.sample(1000)
data_1000_sample.shape
label_col = np.array(data_1000_sample.iloc[:,-1]).reshape(data_1000_sample.shape[0],1)
data_1000_sample.shape

(1000, 2)

In [ ]:
start_time = time.time()

embeddings = []
token_embedding = ELMoEmbeddings()
document_embedding = DocumentPoolEmbeddings([token_embedding])
# giving data_1000_samples as input
for sent in tqdm(data_1000_sample.iloc[:,0]):
  sentence = Sentence(sent)
  document_embedding.embed(sentence)
  embeddings.append(sentence.embedding)

end_time = time.time()
print(f'\n Inference ran for {round((end_time -  start_time))} seconds for 1000 datapoints')
print(f' \n For 1 datapoint inference ran for {round((end_time -  start_time)/1000, 2)} seconds')

100%|██████████| 1000/1000 [07:26<00:00,  2.24it/s]


 Inference ran for 458 seconds for 1000 datapoints
 
 For 1 datapoint inference ran for 0.46 seconds


In [ ]:
device = 'cpu'
embeddings_tensor = torch.stack(embeddings, dim=0).to(device)
hf_elmo = embeddings_tensor.numpy()
hf_elmo.shape

torch.Size([1000, 3072])

In [ ]:
start_time = time.time()

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    clf = LogisticRegression(solver = "lbfgs", random_state = 0)
    clf.fit(hf_elmo, label_col)

end_time = time.time()
print(f'\n Inference ran for {round((end_time -  start_time))} seconds for 1000 datapoints')
print(f' \n For 1 datapoint inference ran for {round((end_time -  start_time)/1000, 2)} seconds')


 Inference ran for 0 seconds for 1000 datapoints
 
 For 1 datapoint inference ran for 0.0 seconds


Adding the Embedding extraction inference time with logistic regression inference time we get 0.45 seconds for 1 data point